In [52]:
import os
import sys
import django

sys.path.insert(0, os.path.abspath('..'))
os.environ['DJANGO_SETTINGS_MODULE'] = 'wui.settings'
os.environ['SECRET_KEY'] = '12345'
django.setup()

In [53]:
def convert_index(index):
    """ Convert A/C into R (red) and T into G (green). """
    return re.sub('T', 'G', re.sub('A|C', 'R', index))


def best_pair_scores(index1, index2, depth1, depth2):
    """
    Calculate the score for two given indices.

    Score is an absolute difference between the sequence depths of
    the two indices divided by the total sequence depth (in %).

    The ideal score is 0.0 (50% green and 50% red),
    an acceptable score is 60.0 (80%/20% or 20%/80%).

    If the score > 60%, then the indices are not compatible.
    """
    index1 = convert_index(index1)
    index2 = convert_index(index2)
    total_depth = depth1 + depth2
    result = []

    for cycle in range(6):
        if index1[cycle] != index2[cycle]:
            difference = abs(depth1 - depth2) / total_depth * 100
            if difference <= 60.0:
                result.append(difference)
            else:
                # If color diversity is low (81%/19% or worse) in
                # one of the cycles
                return -1
        else:
            # If nucleotides are only red (A/C) or green (G/T) in a cycle
            return -1

    return result

In [54]:
import re

from library_sample_shared.models import IndexI7, IndexI5
from library.models import Library
from sample.models import Sample

# library_ids = [4, 6]
library_ids = []
sample_ids = [1, 2, 3]

In [60]:
libraries = Library.objects.filter(id__in=library_ids)
samples = Sample.objects.filter(id__in=sample_ids)

result = []


# Index types for all libraries and samples
index_types = list(set(
    [l.index_type for l in libraries] + [s.index_type for s in samples]
))

# Dictionary with all indices
indices = {}
for index_type in index_types:
    indices[index_type.pk] = {'i7': [], 'i5': []}
    
    if index_type.is_index_i7:
        indices[index_type.pk]['i7'].extend([
            {
                'index': idx.index,
                'index_id': idx.pk,
            }
            for idx in IndexI7.objects.filter(index_type=index_type.pk)
        ])
        
        indices[index_type.pk]['i5'].extend([
            {
                'index': idx.index,
                'index_id': idx.pk,
            }
            for idx in IndexI5.objects.filter(index_type=index_type.pk)
        ])


num_libraries = libraries.count()
num_samples = samples.count()


if num_samples > 0:
    if num_libraries > 0:
        # Add all libraries directly to the result
        case = 1
    else:
        # Generate indices only for samples
        case = 2

        if num_samples > 1:
            # Find a pair of best matching indices (I7)
            # TODO@me: ensure index_type != ''
            best_pair = {'avg_score': 100.0}  # the worst score
            for index1 in indices[samples[0].index_type.pk]['i7']:
                for index2 in indices[samples[1].index_type.pk]['i7']:
                    if index1 != index2:
                        scores = best_pair_scores(
                            index1['index'],
                            index2['index'],
                            samples[0].sequencing_depth,
                            samples[1].sequencing_depth
                        )
                        
                        # If both indices are compatible
                        if scores != -1:
                            avg_score = sum(scores) / 6
                            if avg_score < best_pair['avg_score']:
                                best_pair = {
                                    'index1': index1,
                                    'index2': index2,
                                    'avg_score': avg_score
                                }
            if 'index1' in best_pair.keys():
                pass
            else:
                error = 'Could not find the best matching pair of indices.'
        else:
            error = 'It is not possible to generate an index only for one sample.'
else:
    error = 'No samples.'

if error == '':
    if case == 1:
        # If there are any libraries, consider all samples 
        start = 0
    else:
        # case 2: generate indices for all samples, except for the first two
        start = 2
        
    

{'index1': {'index_id': 73, 'index': 'TAAGGCGA'}, 'avg_score': 0.0, 'index2': {'index_id': 69, 'index': 'CGTACG'}}


<QuerySet [<Library: Library3>, <Library: Library5>]>